In [1]:
from bokeh.io import output_notebook, output_file, show, save
from bokeh.plotting import figure
from bokeh.layouts import row
from bokeh.palettes import Dark2_5 as palette
from bokeh.models import Range1d

import pandas as pd
import numpy as np
from scipy.stats import mode
from glob import glob

output_notebook()
# output_file('throughput-latency-curves.html')

Loading BokehJS ...

In [2]:
filenames = glob('logs/*.csv')
get_name = lambda fname: fname.split('-')[0].split('_', maxsplit=1)[1]
model_names = list(set(map(get_name, filenames)))

x_col = 'p50 latency'
y_col = 'Inferences/Second'
batch_size = 8
min_y, max_y = 1<<25, 0

figures = []
for i, instances in enumerate([1, 2, 4]):
    p = figure(
        height=500,
        width=350,
        tools='',
        title='{} Concurrent Model Instances'.format(instances)
    )
    p.xaxis.axis_label = 'p50 Latency (us)'
    p.yaxis.axis_label = 'Throughput (Samples/Second)'

    sort_col = 'Concurrency'
    read_csv = lambda f: pd.read_csv(
        f, usecols=[sort_col, x_col, y_col], index_col=sort_col).sort_values(
        by=sort_col).values
    for model_name, color in zip(model_names, palette):
        # first one isn't full for some reason, going to just hack
        csvs = glob('logs/deepclean_{}-{}-*.csv'.format(model_name, instances))
        dfs = list(map(read_csv, csvs))

        # some of these can be short if something went wrong, we'll
        # just trust that most of them went right and get rid of the others
        lens = list(map(len, dfs))
        dfs = list(filter(lambda df: len(df) == mode(lens).mode[0], dfs))
        data = sum(dfs) / len(dfs)
        y, x = data.T

        p.line(
            x,
            batch_size*y,
            line_color=color,
            legend_label=model_name,
            line_width=2.3
        )

        min_y = min(min_y, batch_size*y.min())
        max_y = max(max_y, batch_size*y.max())

    p.legend.location = 'bottom_right'
    figures.append(p)

for p in figures:
    p.y_range = Range1d(0.95*min_y, 1.05*max_y)

show(row(figures))
# save(row(figures))